In [1]:
import torch

print("Test for GPU!")
print(torch.cuda.is_available())

Test for GPU!
True


# Load Dataset and Model

In [2]:
from datasets import load_dataset

# Load Dataset
dataset_name = "glue"
task_name = "sst2"
dataset = load_dataset(dataset_name, task_name, split="train")
validation = load_dataset(dataset_name, task_name, split="validation")

Found cached dataset glue (/home/9130/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Found cached dataset glue (/home/9130/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline

# Version 2-13b-chat
base_model_name = "meta-llama/Llama-2-13b-chat-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

device_map = {"": 0}

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map=device_map,
    trust_remote_code=True,
    use_auth_token=True
)

base_model.config.use_cache = False

# More info: https://github.com/huggingface/transformers/pull/24906
base_model.config.pretraining_tp = 1 

/usr/local/lib/python3.10/site-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"
tokenizer

LlamaTokenizerFast(name_or_path='meta-llama/Llama-2-13b-chat-hf', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False), 'pad_token': '</s>'}, clean_up_tokenization_spaces=False)

# Inference

## Create Prompt Template

### Few-shot Inference by Text-Generation

In [76]:
# Define few-shot samples
examples = [
    {
        "instruction": "Classify the sentiment of the following text only into these two categories : 'positive' or 'negative':",
        "sentence": "comes from the brave , uninhibited performances",
        "label": "positive",
    },
    {
        "instruction": "Classify the sentiment of the following text only into these two categories : 'positive' or 'negative':",
        "sentence": "a depressed fifteen-year-old 's suicidal poetry",
        "label": "negative",
    },
    {
        "instruction": "Classify the sentiment of the following text only into these two categories : 'positive' or 'negative':",
        "sentence": "it 's about issues most adults have to face in marriage and i think that 's what i liked about it -- the real issues tucked between the silly and crude storyline",
        "label": "positive",
    },
    {
        "instruction": "Classify the sentiment of the following text only into these two categories : 'positive' or 'negative':",
        "sentence": "will find little of interest in this film , which is often preachy and poorly acted",
        "label": "negative",
    },
]

In [77]:
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate

# Define example prompt and few-shot prompt
example_prompt = PromptTemplate(
    input_variables=["instruction", "sentence", "label"], 
    template="{instruction}\nSentence: {sentence}\nSentiment: {label}",
)

prompt = FewShotPromptTemplate(
    examples=examples, 
    example_prompt=example_prompt, 
    suffix="Classify the sentiment of the following text only into these two categories :'positive' or 'negative':\nSentence: {sentence}\nSentiment: ", 
    input_variables=["sentence"]
)


### Simple Test

In [78]:
sent2test = "I am so happy!"
prompt2test = prompt.format(sentence=sent2test)
print(prompt2test)

Classify the sentiment of the following text only into these two categories : 'positive' or 'negative':
Sentence: comes from the brave , uninhibited performances
Sentiment: positive

Classify the sentiment of the following text only into these two categories : 'positive' or 'negative':
Sentence: a depressed fifteen-year-old 's suicidal poetry
Sentiment: negative

Classify the sentiment of the following text only into these two categories : 'positive' or 'negative':
Sentence: it 's about issues most adults have to face in marriage and i think that 's what i liked about it -- the real issues tucked between the silly and crude storyline
Sentiment: positive

Classify the sentiment of the following text only into these two categories : 'positive' or 'negative':
Sentence: will find little of interest in this film , which is often preachy and poorly acted
Sentiment: negative

Classify the sentiment of the following text only into these two categories :'positive' or 'negative':
Sentence: I am 

In [79]:
inputs = tokenizer(prompt2test, return_tensors="pt").to("cuda")
outputs = base_model.generate(input_ids=inputs["input_ids"].to("cuda"), attention_mask=inputs["attention_mask"], max_new_tokens=80, pad_token_id=tokenizer.eos_token_id)

print(tokenizer.decode(outputs[0], skip_special_tokens=False))

<s> Classify the sentiment of the following text only into these two categories : 'positive' or 'negative':
Sentence: comes from the brave , uninhibited performances
Sentiment: positive

Classify the sentiment of the following text only into these two categories : 'positive' or 'negative':
Sentence: a depressed fifteen-year-old 's suicidal poetry
Sentiment: negative

Classify the sentiment of the following text only into these two categories : 'positive' or 'negative':
Sentence: it 's about issues most adults have to face in marriage and i think that 's what i liked about it -- the real issues tucked between the silly and crude storyline
Sentiment: positive

Classify the sentiment of the following text only into these two categories : 'positive' or 'negative':
Sentence: will find little of interest in this film , which is often preachy and poorly acted
Sentiment: negative

Classify the sentiment of the following text only into these two categories :'positive' or 'negative':
Sentence: I

### Inference and Evaluate

In [65]:
import re
from tqdm import tqdm

def evaluate_text_generation(dataset, prompt, batch_size=16):
    label_map = {
        0 : 'negative',
        1 : 'positive',
    }

    chatDataset = ChatDataset(dataset, tokenizer, label_map, prompt)
    data_loader = DataLoader(chatDataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

    compared_result = []
    invalid_label = []

    for i, batch in enumerate(tqdm(data_loader)):
        # Move batch to GPU
        input_ids = batch["input_ids"].to("cuda")
        attention_mask = batch["attention_mask"].to("cuda")
        labels = batch["labels"].to("cuda")

        # Generate for the entire batch
        outputs = base_model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=80,
            pad_token_id=tokenizer.eos_token_id
        )

        # Decode the generated text and labels
        outputs_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        label_decoded = tokenizer.batch_decode(labels, skip_special_tokens=True)

        # print("output 0")
        # print(outputs_text[0])
        # print("output 1")
        # print(outputs_text[1])
        # print("output 2")
        # print(outputs_text[2])
        # assert 1 == 0

        # Evaluate the generated text
        for idx in range(len(outputs_text)):
            # Extract the last sentence
            selected_sentiment = outputs_text[idx].split("\n")[-1].lower()
            # Remove the prompt
            selected_sentiment = selected_sentiment.split(" ")[-1]

            # Abnormal case
            if selected_sentiment not in ['positive', 'negative']:
                invalid_label.append(selected_sentiment)
                compared_result.append(0)
                continue
            
            if selected_sentiment == label_decoded[idx]:
                compared_result.append(1)
            else:
                compared_result.append(0)

        

    return compared_result, invalid_label


In [80]:
compared_result, invalid_label = evaluate_text_generation(validation, prompt, 32)

100%|██████████| 28/28 [01:26<00:00,  3.09s/it]


In [81]:
showEvalResults(compared_result, invalid_label)

Accuracy: 0.9392201834862385
# of Invalid labels: 1 out of 872 samples
Invalid labels: Counter({'neutral': 1})


### Few-shot Inference by single-turn Chat

In [19]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

# Define examples
examples = [
    {
        "instruction": "Classify the sentiment of the following text only into these two categories : 'positive' or 'negative':",
        "sentence": "comes from the brave , uninhibited performances",
        "label": "positive",
    },
    {
        "instruction": "Classify the sentiment of the following text only into these two categories : 'positive' or 'negative':",
        "sentence": "a depressed fifteen-year-old 's suicidal poetry",
        "label": "negative",
    },
]

In [32]:
example_prompt = PromptTemplate(
    input_variables=["instruction", "sentence", "label"], 
    template="{instruction}\nSentence: {sentence}\nSentiment: {label}",
)

prompt = FewShotPromptTemplate(
    examples=examples, 
    example_prompt=example_prompt, 
    suffix="Classify the sentiment of the following text only into these two categories :'positive' or 'negative':\nSentence: {sentence}\nSentiment: </INST>", 
    input_variables=["sentence"]
)

prompt_prefix = "<s><INST> "

In [33]:
sent2test = "I am so happy!"
prompt2test = prompt_prefix + prompt.format(sentence=sent2test)
print(prompt2test)

<s><INST> Classify the sentiment of the following text only into these two categories : 'positive' or 'negative':
Sentence: comes from the brave , uninhibited performances
Sentiment: positive

Classify the sentiment of the following text only into these two categories : 'positive' or 'negative':
Sentence: a depressed fifteen-year-old 's suicidal poetry
Sentiment: negative

Classify the sentiment of the following text only into these two categories :'positive' or 'negative':
Sentence: I am so happy!
Sentiment: </INST>


In [34]:
inputs = tokenizer(prompt2test, return_tensors="pt").to("cuda")
outputs = base_model.generate(input_ids=inputs["input_ids"].to("cuda"), attention_mask=inputs["attention_mask"], max_new_tokens=80, pad_token_id=tokenizer.eos_token_id)

print(tokenizer.decode(outputs[0], skip_special_tokens=False))

<s><s> <INST> Classify the sentiment of the following text only into these two categories : 'positive' or 'negative':
Sentence: comes from the brave , uninhibited performances
Sentiment: positive

Classify the sentiment of the following text only into these two categories : 'positive' or 'negative':
Sentence: a depressed fifteen-year-old 's suicidal poetry
Sentiment: negative

Classify the sentiment of the following text only into these two categories :'positive' or 'negative':
Sentence: I am so happy!
Sentiment: </INST>  Sure! Here are the classifications for each sentence:

1. "comes from the brave, uninhibited performances" - Positive
2. "a depressed fifteen-year-old's suicidal poetry" - Negative
3. "I am so happy!" - Positive</s>


### Few-shot Inference by multi-turn Chat

In [7]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

# Define examples
examples = [
    {
        "instruction": "Classify the sentiment of the following text only into these two categories : 'positive' or 'negative':",
        "sentence": "comes from the brave , uninhibited performances",
        "label": "positive",
    },
    {
        "instruction": "Classify the sentiment of the following text only into these two categories : 'positive' or 'negative':",
        "sentence": "a depressed fifteen-year-old 's suicidal poetry",
        "label": "negative",
    },
    # {
    #     "instruction": "Classify the sentiment of the following text only into these two categories : 'positive' or 'negative':",
    #     "sentence": "it 's about issues most adults have to face in marriage and i think that 's what i liked about it -- the real issues tucked between the silly and crude storyline",
    #     "label": "positive",
    # },
    # {
    #     "instruction": "Classify the sentiment of the following text only into these two categories : 'positive' or 'negative':",
    #     "sentence": "will find little of interest in this film , which is often preachy and poorly acted",
    #     "label": "negative",
    # },
]


In [8]:
# Define example prompt and few-shot prompt
example_prompt = PromptTemplate(
    input_variables=["instruction", "sentence", "label"], 
    template="<s><INST> {instruction}\nSentence: {sentence}\nSentiment: </INST> {label} </s>",
)

prompt = FewShotPromptTemplate(
    examples=examples, 
    example_prompt=example_prompt, 
    suffix="<s><INST> Classify the sentiment of the following text only into these two categories :'positive' or 'negative':\nSentence: {sentence}\nSentiment: </INST>", 
    input_variables=["sentence"]
)

### Simple Test

In [9]:
sent2test = "I am so happy!"
prompt2test = prompt.format(sentence=sent2test)
print(prompt2test)

<s><INST> Classify the sentiment of the following text only into these two categories : 'positive' or 'negative':
Sentence: comes from the brave , uninhibited performances
Sentiment: </INST> positive </s>

<s><INST> Classify the sentiment of the following text only into these two categories : 'positive' or 'negative':
Sentence: a depressed fifteen-year-old 's suicidal poetry
Sentiment: </INST> negative </s>

<s><INST> Classify the sentiment of the following text only into these two categories :'positive' or 'negative':
Sentence: I am so happy!
Sentiment: </INST>


In [10]:
inputs = tokenizer(prompt2test, return_tensors="pt").to("cuda")
outputs = base_model.generate(input_ids=inputs["input_ids"].to("cuda"), attention_mask=inputs["attention_mask"], max_new_tokens=80, pad_token_id=tokenizer.eos_token_id)

print(tokenizer.decode(outputs[0], skip_special_tokens=False))

<s><s> <INST> Classify the sentiment of the following text only into these two categories : 'positive' or 'negative':
Sentence: comes from the brave , uninhibited performances
Sentiment: </INST> positive </s> 

<s> <INST> Classify the sentiment of the following text only into these two categories : 'positive' or 'negative':
Sentence: a depressed fifteen-year-old 's suicidal poetry
Sentiment: </INST> negative </s> 

<s> <INST> Classify the sentiment of the following text only into these two categories :'positive' or 'negative':
Sentence: I am so happy!
Sentiment: </INST> positive</s>


### Inference and evaluate

In [11]:
import torch
from torch.utils.data import Dataset, DataLoader

class ChatDataset(Dataset):
    def __init__(self, dataset, tokenizer, label_map, prompt):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.label_map = label_map
        self.prompt = prompt

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        val = self.dataset[idx]
        label_text = self.label_map[val['label']]
        sentence = val['sentence'][:-1]
        text = self.prompt.format(sentence=sentence)
        inputs = self.tokenizer(text, return_tensors="pt").to("cuda")
        labels = self.tokenizer(label_text, return_tensors="pt").to("cuda")
        
        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze(),
            'labels': labels['input_ids'].squeeze()
        }


In [12]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    input_ids = [item['input_ids'].tolist() for item in batch]
    attention_mask = [item['attention_mask'].tolist() for item in batch]
    labels = [item['labels'] for item in batch]

    # Left Padding
    max_length = max([len(item) for item in input_ids])
    input_ids = [[0]*(max_length - len(item)) + item for item in input_ids]
    attention_mask = [[0]*(max_length - len(item)) + item for item in attention_mask]

    # Convert lists to tensors
    input_ids = torch.tensor(input_ids)
    attention_mask = torch.tensor(attention_mask)
    # Usually, labels are not padded
    labels = pad_sequence(labels, batch_first=True, padding_value=-100)

    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': labels
    }

In [13]:
import re
from tqdm import tqdm

def evaluate_chat(dataset, prompt, batch_size=16):
    label_map = {
        0 : 'negative',
        1 : 'positive',
    }

    chatDataset = ChatDataset(dataset, tokenizer, label_map, prompt)
    data_loader = DataLoader(chatDataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

    compared_result = []
    invalid_label = []

    for i, batch in enumerate(tqdm(data_loader)):
        # Move batch to GPU
        input_ids = batch["input_ids"].to("cuda")
        attention_mask = batch["attention_mask"].to("cuda")
        labels = batch["labels"].to("cuda")

        # Generate for the entire batch
        outputs = base_model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=80,
            pad_token_id=tokenizer.eos_token_id
        )

        # Decode the generated text and labels
        outputs_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        label_decoded = tokenizer.batch_decode(labels, skip_special_tokens=True)

        # Evaluate the generated text
        for idx in range(len(outputs_text)):
            # Extract the last sentence
            selected_sentiment = outputs_text[idx].split("\n")[-1].lower()
            # Remove the prompt
            selected_sentiment = selected_sentiment.split(" ")[-1]
            if selected_sentiment not in ['positive', 'negative']:
                invalid_label.append(selected_sentiment)
                compared_result.append(0)
                continue
            
            if selected_sentiment == label_decoded[idx]:
                compared_result.append(1)
            else:
                compared_result.append(0)

    return compared_result, invalid_label


In [14]:
comp_res, invalid_label = evaluate_chat(validation, prompt, batch_size=32)

100%|██████████| 28/28 [00:32<00:00,  1.16s/it]


In [17]:
from collections import Counter
def showEvalResults(compare_results, invalid_label):
    counted_elements = Counter(invalid_label)
    accuracy = compare_results.count(1)/len(compare_results)
    print("Accuracy:", accuracy)
    print("# of Invalid labels:", len(invalid_label), "out of", len(compare_results), "samples")
    print("Invalid labels:", counted_elements)


In [18]:
showEvalResults(comp_res, invalid_label)

Accuracy: 0.9243119266055045
# of Invalid labels: 9 out of 872 samples
Invalid labels: Counter({'neutral': 8, 'mixed': 1})
